Universidade Federal de Santa Catarina<br>
Departamento de Engenharia Elétrica e Eletrônica<br>
EEL7514/EEL7513 - Introdução ao Aprendizado de Máquina
$\newcommand{\bX}{\mathbf{X}}$
$\newcommand{\bw}{\mathbf{w}}$
$\newcommand{\by}{\mathbf{y}}$
$\newcommand{\bx}{\mathbf{x}}$
$\newcommand{\bW}{\mathbf{W}}$
$\newcommand{\bb}{\mathbf{b}}$
$\newcommand{\ba}{\mathbf{a}}$
$\newcommand{\bz}{\mathbf{z}}$
$\newcommand{\bA}{\mathbf{A}}$
$\newcommand{\bZ}{\mathbf{Z}}$
$\newcommand{\bS}{\mathbf{S}}$
$\newcommand{\bL}{\mathbf{L}}$
$\newcommand{\bR}{\mathbf{R}}$
$\newcommand{\bdelta}{\boldsymbol{\delta}}$
$\newcommand{\bDelta}{\boldsymbol{\Delta}}$
$\newcommand{\calD}{\mathcal{D}}$
$\newcommand{\mat}[1]{\begin{bmatrix} #1 \end{bmatrix}}$


# Exercício 9: Redes Profundas


Neste exercício, você irá explorar várias técnicas importantes para o treinamento de redes profundas:
- Função de ativação ReLU
- Variações do método do gradiente estocástico
- Batch normalization
- Dropout

Lembre que, além da implementação, você deve incluir também uma **análise** dos resultados. 

# Instruções gerais
O exercício consiste em realizar as partes Q1, Q2 e Q3 do exercício:

http://cs231n.github.io/assignments2018/assignment2/

exceto a seção "Layer Normalization" da parte Q2.

Obs: embora os slides de referência sejam do oferecimento de 2017, para o exercício é recomendável utilizar a versão mais atualizada, de 2018. Os slides de 2018 também podem ser consultados, ignorando o conteúdo novo sobre "Layer Normalization" (e, por hora, qualquer menção a redes convolucionais).

## Instalação

Ao rodar a primeira célula da Q1 do exercício, você observará alguns erros e warnings. Para corrigi-los, siga os seguintes passos:
- Do arquivo `cs231n/gradient_check.py`, remova a linha 
```python
from past.builtins import xrange
```
- Do arquivo `cs231n/solver.py`, remova as linhas 
```python
from future import standard_library
standard_library.install_aliases()
```
- Ignore o warning "_run the following from the cs231n directory and try again_"


# Observações

A notação usada no exercício é frequentemente inconsistente com a notação usada nos slides da disciplina CS231n. Por exemplo, a matriz $\bW$ aparece nos slides seguindo a notação que adotamos em sala, mas no exercício aparece transposta. É preciso ter cuidado para interpretar corretamente a orientação das matrizes, o que normalmente pode ser obtido analisando suas dimensões.

Além disso, no exercício, todos os vetores são representados como vetores 1-D no NumPy, o que significa que são difundidos (broadcast) como vetores-linha. Vale a pena se acostumar com essa notação, pois ela de fato facilita a implementação.

Várias funções por padrão utilizam a variável `x` para denotar a "entrada" da função, o que não necessiariamente significa um vetor de atributos.

O termo _score_ é usado para denotar a variável $\bz^{[L]}$, isto é, a saída da última camada antes da aplicação da função de ativação (no caso, softmax). A ideia por trás é que, em qualquer classificador binário ou um-contra-todos, a predição pode ser feita de acordo com o valor máximo do _score_, sem necessidade de aplicar a função de ativação. Assim, a função de ativação de saída só possui realmente utilidade para calcular a função perda, que por esse motivo é denominada "perda softmax" (na verdade, entropia cruzada categórica com saída softmax). Mencionamos esse fato para a ativação logística no slide 3 da Aula 6.

Ao contrário dos nossos slides, o exercício não utiliza divisão por $m$ no cálculo da função custo (nem para a perda softmax nem para a regularização).

Alguns exemplos de diferença de notação:
- Slides (EEL7514) $\longrightarrow$ Exercício
- Número de amostras: $m \longrightarrow N$
- Número de atributos: $n \longrightarrow D$

Para exemplificar (e considerando que esse conteúdo inicial já foi coberto pelo Exercício 5), as soluções das duas primeiras tarefas são fornecidas abaixo:

```python
def affine_forward(x, w, b):
    out = x.reshape(x.shape[0],-1) @ w + b
    cache = (x, w, b)
    return out, cache

def affine_backward(dout, cache):
    x, w, b = cache
    dx = (dout @ w.T).reshape(x.shape)
    dw = x.reshape(x.shape[0],-1).T @ dout
    db = dout.sum(axis=0)
    return dx, dw, db
```